In [ ]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
from nst_utils import * #nst_utils.py文件中定义了图片预处理、生成白躁图等函数
import numpy as np
import tensorflow as tf

%matplotlib inline ##使用%matplotlib命令可以将matplotlib的图表直接嵌入到Notebook之中，实验必备


### 定义代价函数（内容损失+风格损失）


**Instructions**

- 获得某个 tensor X的维数: `X.get_shape().as_list()`
- [tf.transpose用法](https://www.tensorflow.org/versions/r1.3/api_docs/python/tf/transpose) 及 [tf.reshape用法](https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/reshape).
- [tf.reduce_sum用法](https://www.tensorflow.org/api_docs/python/tf/reduce_sum).

In [ ]:
# 计算内容损失

def compute_content_cost(a_C, a_G):
    """
    Computes the content cost
    
    Arguments:
    a_C -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image C 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image G
    
    Returns: 
    J_content -- scalar that you compute using equation 1 above.
    """
    
    # 获得a_G维数 
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
#     print("m:{0}".format(m))
#     print("n_H:{0}".format(n_H))
#     print("n_W:{0}".format(n_W))
#     print("n_C:{0}".format(n_C))

    # Reshape a_C and a_G 
    a_C_unrolled = tf.reshape(tf.transpose(a_C, perm=[3, 2, 1, 0]), [n_C, n_H*n_W, -1])
    a_G_unrolled = tf.reshape(tf.transpose(a_G, perm=[3, 2, 1, 0]), [n_C, n_H*n_W, -1])
    # compute the cost with tensorflow 
    J_content = tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled, a_G_unrolled))) / (4 * n_H * n_W * n_C)
    
    return J_content

In [2]:
# 定义Gram矩阵

def gram_matrix(A):
    """
    Argument:
    A -- matrix of shape (n_C, n_H*n_W)
    
    Returns:
    GA -- Gram matrix of A, of shape (n_C, n_C)
    """
    
    GA = tf.matmul(A,tf.transpose(A)) #作内积
    return GA

In [ ]:
# 计算一层的风格损失

def compute_layer_style_cost(a_S, a_G):
    """
    Arguments:
    a_S -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image S 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image G
    
    Returns: 
    J_style_layer -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    # Retrieve dimensions from a_G 
    m, n_H, n_W, n_C = a_G.get_shape().as_list()

    # Reshape the images to have them of shape (n_C, n_H*n_W) 
    a_S = tf.reshape(tf.transpose(a_S, perm=[3, 1, 2, 0]), [n_C, n_W*n_H])
    a_G = tf.reshape(tf.transpose(a_G, perm=[3, 1, 2, 0]), [n_C, n_W*n_H])

    # Computing gram_matrices for both images S and G 
    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)

    # Computing the loss 
    J_style_layer = tf.reduce_sum(tf.square(tf.subtract(GS, GG))) / (4 * n_C**2 * (n_W * n_H)**2)


    
    return J_style_layer

In [3]:
# 从以下几个层中计算风格损失，数字代表权重

STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)]

In [ ]:
# 计算多层风格损失

def compute_style_cost(model, STYLE_LAYERS):
    """
    Computes the overall style cost from several chosen layers
    
    Arguments:
    model -- our tensorflow model
    STYLE_LAYERS -- A python list containing:
                        - the names of the layers we would like to extract style from
                        - a coefficient for each of them
    
    Returns: 
    J_style -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    # initialize the overall style cost
    J_style = 0

    for layer_name, coeff in STYLE_LAYERS:

        # Select the output tensor of the currently selected layer
        out = model[layer_name]

        # Set a_S to be the hidden layer activation from the layer we have selected, by running the session on out
        a_S = sess.run(out)

        # Set a_G to be the hidden layer activation from same layer. Here, a_G references model[layer_name] 
        # and isn't evaluated yet. Later in the code, we'll assign the image G as the model input, so that
        # when we run the session, this will be the activations drawn from the appropriate layer, with G as input.
        a_G = out
        
        # Compute style_cost for the current layer
        J_style_layer = compute_layer_style_cost(a_S, a_G)

        # Add coeff * J_style_layer of this layer to overall style cost
        J_style += coeff * J_style_layer

    return J_style

In [4]:
# 计算总的损失

def total_cost(J_content, J_style, alpha = 10, beta = 40):
    """
    Computes the total cost function
    
    Arguments:
    J_content -- content cost coded above
    J_style -- style cost coded above
    alpha -- hyperparameter weighting the importance of the content cost
    beta -- hyperparameter weighting the importance of the style cost
    
    Returns:
    J -- total cost as defined by the formula above.
    """
    
    J = alpha*J_content + beta*J_style
    
    
    return J

### 生成图片

**Instructions**

- [Interactive Session](https://www.tensorflow.org/api_docs/python/tf/InteractiveSession)是tf的一个类，可以省略掉重复指向对象的过程
- 迁移学习，利用预训练好的VGG-19模型进行特征提取
- [Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer)优化生成图片（图像重建）

In [ ]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()

In [ ]:
# 读入内容、风格图片
content_image = scipy.misc.imread("images/louvre_small.jpg")
content_image = reshape_and_normalize_image(content_image)
style_image = scipy.misc.imread("images/monet.jpg")
style_image = reshape_and_normalize_image(style_image)

In [ ]:
# 生成白躁图
generated_image = generate_noise_image(content_image)
imshow(generated_image[0])

# 加入预训练好的VGG-19网络
model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19.mat")

# 将内容图片作为VGG网络的输入  
sess.run(model['input'].assign(content_image))

# 选取要提取特征的层conv4_2
out = model['conv4_2']
a_C = sess.run(out)
a_G = out 

# 计算内容损失
J_content = compute_content_cost(a_C, a_G)

# 将风格图片作为VGG网络的输入  
sess.run(model['input'].assign(style_image))

# 计算风格损失
J_style = compute_style_cost(model, STYLE_LAYERS)

# 计算总的损失
J = total_cost(J_content, J_style, alpha = 10, beta = 40)

In [ ]:
# 定义优化器AdamOptimizer
optimizer = tf.train.AdamOptimizer(2.0)

# 定义训练步长
train_step = optimizer.minimize(J)

In [ ]:
# 定义整个神经网络的计算：初始化参数，输入图片，多次迭代优化生成图片
def model_nn(sess, input_image, num_iterations = 200):
    
    # Initialize global variables (you need to run the session on the initializer)
    ### START CODE HERE ### (1 line)
    sess.run(tf.global_variables_initializer())
    ### END CODE HERE ###

    # Run the noisy input image (initial generated image) through the model. Use assign().
    ### START CODE HERE ### (1 line)
    sess.run(model['input'].assign(input_image))
    ### END CODE HERE ###

    for i in range(num_iterations):

        # Run the session on the train_step to minimize the total cost
        ### START CODE HERE ### (1 line)
        sess.run(train_step)
        ### END CODE HERE ###

        # Compute the generated image by running the session on the current model['input']
        ### START CODE HERE ### (1 line)
        generated_image = sess.run(model['input'])
        ### END CODE HERE ###

        # Print every 20 iteration.
        if i%20 == 0:
            Jt, Jc, Js = sess.run([J, J_content, J_style])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))

            # save current generated image in the "/output" directory
            save_image("output/" + str(i) + ".png", generated_image)

    # save last generated image
    save_image('output/generated_image.jpg', generated_image)

    return generated_image

In [ ]:
# 下面这行代码，用CPU要跑3min左右。但在100多次迭代后（每20次迭代后输出一张生成图）基本上就不会有太多变化了。
model_nn(sess, generated_image)